# Establish Authenticated Connection's with Data Scientist's

In this notebook you will learn how to set up an entry point for data scientists who wish to connect and access your data for their privacy-preserving machine learning flow. All connections through this entrypoint are challenged with an authentication policy that you will define, only once this policy has been met through a verifiable presentation will the connection become trusted.

You can imagine a list of trusted connection's might have certain capabilities to interact with the data owner and it's data that untrusted connections would not. By the end of this notebooks it should be easy to see how these connections could further be categorised based on the contents of the attribute that was issued. This is outside the scope of this series.

## 1. Create a Data Owner class

Up until now we have been mainly interacting with the our agent using the basic controller by executing cells in notebooks. The notebook cells combined with our human input have made up the business logic of the SSI application.

However, as a Data Owner we might want to be able to trust connections without requiring manual input if they can meet a pre defined authentication policy.

To define this logic we will create a Data Owner class.

We can either extend the basic controller class, or initialise the agent_controller as a property of the data owner. I am not sure which way is optimal, for now we will extend the class.

In [1]:
%autoawait
import time
import asyncio
import json
# I think this is because jupyter notebook also runs an event loop
import nest_asyncio
nest_asyncio.apply()
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    

class DataOwner(AriesAgentController):
    
    def __init__(self, webhook_host: str, webhook_port: int, admin_url: str, webhook_base: str = ""):
        # Call the AriesAgentController constructor
        super().__init__(webhook_host, webhook_port, admin_url, webhook_base)
        
        # Keep track of which connections the data owner trusts
        self.trusted_scientist_ids = []
        
        # We will set an authentication policy later
        self.auth_policy = None
        
        # We will use this list to keep track of scientists
        self.pending_scientist_connections = []
        
        # Define listener array
        self.agent_listeners = [{"topic":"connections", "handler": self._connections_handler}, 
                          {"topic":"present_proof", "handler": self._proof_handler}]
        
        # Start webhook server in AriesAgentController
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.listen_webhooks())
        
        # Register listeners
        print("Register Listeners")
        print(self.agent_listeners)
        self.register_listeners(self.agent_listeners)
        
        
    # When scientist connections move to active, challenge with authentication policy if set
    def _connections_handler(self, payload):
        loop = asyncio.get_event_loop()
        print("Connection Handler Called")
        connection_id = payload["connection_id"]
        state = payload["state"]
        print(f"Connection {connection_id} in State {state}")
        # Check connection entered through scientist entry point
        for connection in self.pending_scientist_connections:
            if connection["connection_id"] == connection_id:
                if state == "active":

                    print("Pending connection moved to active.")
                    if self.auth_policy:
                        print("\nChallenging with Auth Policy\n")
                        print(self.auth_policy)
                        loop = asyncio.get_event_loop()
                        
                        # Specify the connection id to send the authentication request to
                        proof_request_web_request = {
                            "connection_id": connection_id,
                            "proof_request": self.auth_policy,
                            "trace": False
                        }
                        
                        # Send proof request
                        response = loop.run_until_complete(self.proofs.send_request(proof_request_web_request))
                    else:
                        # No Auth Policy set so trust all scientists
                        print("No Auth Policy set")
                        # Complete future
                        connection["is_trusted"].set_result(True)
                        
                break


    def _proof_handler(self, payload):
        role = payload["role"]
        connection_id = payload["connection_id"]
        pres_ex_id = payload["presentation_exchange_id"]
        state = payload["state"]
        print("\n---------------------------------------------------------------------\n")
        print("Handle present-proof")
        print("Connection ID : ", connection_id)
        print("Presentation Exchange ID : ", pres_ex_id)
        print("Protocol State : ", state)
        print("Agent Role : ", role)
        print("\n---------------------------------------------------------------------\n")


        loop = asyncio.get_event_loop()
        

        if connection_id in self.trusted_scientist_ids:
            print("Connection is a trusted scientist")
            # Only respond to presentation requests from trusted scientists
            # NOTE: FOR SIMPLICITY WE WILL NOT EXPLAIN THIS PART BUT GIVE IT A SCAN
            # It is automatically handled by the ACA-Py flag (auto-respond-presentation-request)
            # Of course realistic scenarios it is not advisable to respond to all presentation requests by default
            if state == "request_received":
                print("Received Authentication Challenge from Scientist")

                credentials_by_reft = {}
                revealed = {}
                self_attested = {}
                predicates = {}

                # select credentials to provide for the proof
                credentials = loop.run_until_complete(self.proofs.get_presentation_credentials(pres_ex_id))
                print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")
                print(credentials)

                # Note we are working on a friendlier api to abstract this away
                if len(credentials) > 0:
                    reveal_cred = credentials[0]
                    print("\nCredential to reveal\n", reveal_cred)


                    if credentials:
                        for row in credentials:

                            for referent in row["presentation_referents"]:
                                if referent not in credentials_by_reft:
                                    credentials_by_reft[referent] = row

                    for referent in payload["presentation_request"]["requested_attributes"]:
                        if referent in credentials_by_reft:
                            revealed[referent] = {
                                "cred_id": credentials_by_reft[referent]["cred_info"][
                                    "referent"
                                ],
                                "revealed": True,
                            }


                    print("\nGenerate the proof")
                    proof = {
                        "requested_predicates": predicates,
                        "requested_attributes": revealed,
                        "self_attested_attributes": self_attested,
                    }
                    print(proof)
                    print("\nXXX")
                    print(predicates)
                    print(revealed)
                    print(self_attested)

                    loop.run_until_complete(self.proofs.send_presentation(pres_ex_id, proof))
                else:
                    print("\nYour agent does not have the correct credentials. Are you sure you issued yourself it?\n")
        
        else:
            # Only verify presentation's from pending scientist connections
            for connection in self.pending_scientist_connections:
                if connection["connection_id"] == connection_id:
                    print("Connection is a pending scientist")

                    if state == "presentation_received":


                        print("Verifying Presentation from Data Scientist")
                        verify = loop.run_until_complete(self.proofs.verify_presentation(pres_ex_id))
                        # Completing future with result of the verification - True of False
                        connection["is_trusted"].set_result(verify['state'] == "verified")
                    break
            
    def set_auth_policy(self, proof_request):
        self.auth_policy = proof_request

        
        
    
    # Our entry point for scientists
    # Maybe this invitation would be displayed on the Data Owner's website?
    def create_scientist_invite(self):
        loop = asyncio.get_event_loop()
        response = loop.run_until_complete(self.connections.create_invitation())
        connection_id = response["connection_id"]
        invite_message = json.dumps(response['invitation'])

        print()
        print(
            "♫♫♫ > "

            + "STEP 1:"
            + " Copy the aries invitation to the data scientist notebook 7."
        )
        print()
        print(invite_message)
        print()

        pending_connection = {
            "connection_id": connection_id,
            "is_trusted": asyncio.Future()
        }

        self.pending_scientist_connections.append(pending_connection)
        print("Establishing connection")
        # We wait until the is_trusted future is complete
        loop.run_until_complete(pending_connection["is_trusted"])

        # Check is_trusted has evaluated to true
        if pending_connection["is_trusted"].result():

            print(f"Trusted Research Connection Established - {connection_id}")
            print("\n--------------------------------------------------------------------\n")
            print("\n--------------------------------------------------------------------\n")
            print("\n\n")
            self.pending_scientist_connections.remove(pending_connection)
            self.trusted_scientist_ids.append(connection_id)
        

IPython autoawait is `on`, and set to use `asyncio`


## 2. Instantiate the Data Owner

The Data Owner is now an extension of the AriesAgentController and has access to all of it's functions

In [2]:
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://dataowner-agent:8051"

data_owner = DataOwner(WEBHOOK_HOST, WEBHOOK_PORT, ADMIN_URL, WEBHOOK_BASE)

Register Listeners
[{'topic': 'connections', 'handler': <bound method DataOwner._connections_handler of <__main__.DataOwner object at 0x7f8f90579dd0>>}, {'topic': 'present_proof', 'handler': <bound method DataOwner._proof_handler of <__main__.DataOwner object at 0x7f8f90579dd0>>}]


## 3 Set Authentication Policy

The authentication policy is a proof request object that identifies the set of attributes that another agent must present to meet the policy. The proof request object can optionally specify constraints the attributes that must be presented, such as the issuing DID, the schema id and the cred def id.

The below code defines a basic authentication policy for checking Data Scientists, based on the credential schema defined by the OM Authority in part 3.

In [3]:
# We add a constraint that the attribute must originate from this schema
schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1"


# You could additionally specify the cred_def id you wish. 
# You would have to copy this from the OM Authority notebook.
# cred_def = "WfntKNFwXMQfgmU9ofbxPM:3:CL:156569:default"

revocation = False
exchange_tracing = False

# We are only asking the Data Scientist to present the scope attribute from their credential
req_attrs = [
    {"name": "scope", "restrictions": [{"schema_id": schema_id}]},#, "cred_def_id": cred_def}]},
]

# We could extend this to request the name attribute aswell if we wanted.


indy_proof_request = {
    "name": "Proof of Scientist",
    "version": "1.0",
    "requested_attributes": {
        # They must follow this uuid pattern
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}

print(indy_proof_request)



{'name': 'Proof of Scientist', 'version': '1.0', 'requested_attributes': {'0_scope_uuid': {'name': 'scope', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1'}]}}, 'requested_predicates': {}}


In [5]:
# If this is set, the data owner will automatically challenge new connections to prove that they have a Data Scientist credential
# And to disclose the scope attribute issued within that credential.
data_owner.set_auth_policy(indy_proof_request)

## 4. Create Data Scientist Invite

As currently implemented the data scientist invitation is a one time invite. This could be changed to be a multi-use invitation through an argument that can be passed into the create_invitation() function.

It depends on how the application would be using this invitation. Is it displayed statically on the data owner's website? Or does the website expose and button for the data scientist to request an invitation be created?

The user experience and flow of these applications matters, but it is probably the least explored aspect of this technology. If your interested in looking into this with us we would love to have you. Here is an [issue](https://github.com/OpenMined/PyDentity/issues/36) I produced around this, comment if your interested.

In [6]:
data_owner.create_scientist_invite()

Connection Handler Called
Connection fb7528b8-cee2-4938-ab1f-282c86fc05ce in State invitation

♫♫♫ > STEP 1: Copy the aries invitation to the data scientist notebook 7.

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "bd5a48d8-11be-4e17-a880-74e4cea6a45a", "serviceEndpoint": "https://e695090e077b.ngrok.io", "recipientKeys": ["Agp992DQggeQLb17a7LPCn4w64WevuTDr7mkPtnAvn2c"], "label": "DATAOWNER"}

Establishing connection
Connection Handler Called
Connection fb7528b8-cee2-4938-ab1f-282c86fc05ce in State request
Connection Handler Called
Connection fb7528b8-cee2-4938-ab1f-282c86fc05ce in State response
Connection Handler Called
Connection fb7528b8-cee2-4938-ab1f-282c86fc05ce in State active
Pending connection moved to active.

Challenging with Auth Policy

{'name': 'Proof of Scientist', 'version': '1.0', 'requested_attributes': {'0_scope_uuid': {'name': 'scope', 'restrictions': [{'schema_id': 'Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1'}]}}, 'reque

## Continue with Data Scientist

You should copy the connection across to the Data Scientist notebook 7.

## End of Notebook

When you have finished steps 4-7 you will have completed this notebook tutoral. Terminate the controller

In [7]:
await data_owner.terminate()

# Congratulations!!!!
## You have completed this section of the course

In the final lesson we will be applying what we have learnt to authenticate actors in a duet privacy-preserving machine learning flow.